In [4]:
import os
import shutil
from glob import glob

# Utils

In [5]:
def exists(obj):
    return obj is not None

def glob_dirs(regex: str):
    """Returns a list of directories that follow `regex`."""
    return [p for p in glob(regex) if os.path.isdir(p)]

def get_first_match_for_substring(list_, substring: str):
    """Returns the first element in `list_` that matches."""
    match = None
    for elem in list_:
        if substring in elem:
            match = elem
            break
    return match

def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

# Move the patients from ISBI_FOSCAL to MICCAI

In [7]:
src_dir = "/data/Datasets/stroke/ISBI_FOSCAL/"
dest_dir = "/data/Datasets/stroke/MICCAI/"
train_dest_dir = os.path.join(dest_dir, "train")
test_dest_dir = os.path.join(dest_dir, "test")

# Txt files with the patients to be moved to the train and test folders.
train_txt = "/home/sangohe/projects/isbi2023-foscal/splits/train_patients.txt"
test_txt = "/home/sangohe/projects/isbi2023-foscal/splits/test_patients.txt"

with open(train_txt, "r") as f:
    train_patients = f.read().splitlines()
    
    for train_patient in train_patients:
        train_patient_dir = os.path.join(src_dir, train_patient)
        train_patient_dest_dir = os.path.join(train_dest_dir, train_patient)
        os.makedirs(train_patient_dest_dir, exist_ok=True)

        patient_content = glob_dirs(os.path.join(train_patient_dir, "*"))
        adc_dir = get_first_match_for_substring(patient_content, "ADC")
        dwi_dir = get_first_match_for_substring(patient_content, "DWI")
        masks_dir = get_first_match_for_substring(patient_content, "Masks")

        adc_dest_dir = os.path.join(train_patient_dest_dir, os.path.basename(adc_dir))
        dwi_dest_dir = os.path.join(train_patient_dest_dir, os.path.basename(dwi_dir))
        masks_dest_dir = os.path.join(train_patient_dest_dir, os.path.basename(masks_dir))
        
        os.makedirs(adc_dest_dir, exist_ok=True)
        os.makedirs(dwi_dest_dir, exist_ok=True)

        shutil.copy(os.path.join(adc_dir, "ADC.nii.gz"), adc_dest_dir)
        dwi_content = glob(os.path.join(dwi_dir, "*.nii.gz"))
        dwi_match = get_first_match_for_substring(dwi_content, ".nii.gz")
        shutil.copy(dwi_match, os.path.join(dwi_dest_dir, "DWI.nii.gz"))
        copytree(masks_dir, masks_dest_dir, ignore=shutil.ignore_patterns('*.nii', "*NCCT*"))


with open(test_txt, "r") as f:
    test_patients = f.read().splitlines()
    
    for test_patient in test_patients:
        test_patient_dir = os.path.join(src_dir, test_patient)
        test_patient_dest_dir = os.path.join(test_dest_dir, test_patient)
        os.makedirs(test_patient_dest_dir, exist_ok=True)

        patient_content = glob_dirs(os.path.join(test_patient_dir, "*"))
        adc_dir = get_first_match_for_substring(patient_content, "ADC")
        dwi_dir = get_first_match_for_substring(patient_content, "DWI")
        masks_dir = get_first_match_for_substring(patient_content, "Masks")

        adc_dest_dir = os.path.join(test_patient_dest_dir, os.path.basename(adc_dir))
        dwi_dest_dir = os.path.join(test_patient_dest_dir, os.path.basename(dwi_dir))
        masks_dest_dir = os.path.join(test_patient_dest_dir, os.path.basename(masks_dir))
        
        os.makedirs(adc_dest_dir, exist_ok=True)
        os.makedirs(dwi_dest_dir, exist_ok=True)

        shutil.copy(os.path.join(adc_dir, "ADC.nii.gz"), adc_dest_dir)
        dwi_content = glob(os.path.join(dwi_dir, "*.nii.gz"))
        dwi_match = get_first_match_for_substring(dwi_content, ".nii.gz")
        shutil.copy(dwi_match, os.path.join(dwi_dest_dir, "DWI.nii.gz"))
        copytree(masks_dir, masks_dest_dir, ignore=shutil.ignore_patterns('*.nii', "*NCCT*"))